In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt
import pandas as pd
from tqdm import tqdm

In [2]:
# KoNLPy의 Okt 형태소 분석기 사용
okt = Okt()

In [3]:
def tokenize(text):
    # 형태소 분석을 통해 명사만 추출
    return [word for word in okt.nouns(text) if len(word) > 1]

In [4]:
def get_keywords(text, num=10):
    # TF-IDF 벡터화기 생성
    vectorizer = TfidfVectorizer(tokenizer=tokenize, min_df=1)
    X = vectorizer.fit_transform([text])  # 문자열 전체를 하나의 문서로 처리

    # 단어와 TF-IDF 점수 매핑
    tfidf_scores = X.sum(axis=0).A1
    words = vectorizer.get_feature_names_out()

    # 단어와 TF-IDF 점수를 데이터프레임으로 변환
    tfidf_df = pd.DataFrame({'Word': words, 'TF-IDF Score': tfidf_scores})

    # TF-IDF 점수로 정렬하여 상위 num개 키워드 출력
    return tfidf_df.sort_values(by='TF-IDF Score', ascending=False).head(num)

In [5]:
def extract_keywords_from_documents(documents, top_n=10):
    # TF-IDF 벡터화기 생성
    vectorizer = TfidfVectorizer(tokenizer=tokenize, min_df=1)
    X = vectorizer.fit_transform(documents)
    
    # 단어와 TF-IDF 점수 매핑
    words = vectorizer.get_feature_names_out()
    tfidf_scores = X.toarray()
    
    # 각 문서별 키워드 추출
    keywords_per_document = []
    for i, doc_scores in tqdm(enumerate(tfidf_scores)):
        tfidf_df = pd.DataFrame({'Word': words, 'TF-IDF Score': doc_scores})
        top_keywords = tfidf_df.sort_values(by='TF-IDF Score', ascending=False).head(top_n)
        keywords_per_document.append({
            'Document': i,
            'Keywords': ', '.join(top_keywords['Word'])
        })
    
    return pd.DataFrame(keywords_per_document)

In [47]:
def string_to_list(string):
    # 문자열에서 대괄호와 작은따옴표를 제거하고 쉼표로 구분하여 리스트로 변환
    string_list = string.strip("[]").replace("'", "").split(", ")
    
    return string_list

In [48]:
def add_to_set(s1, s2):
    sl1 = string_to_list(s1)
    sl2 = string_to_list(s2)
    
    return set(sl1+sl2)

In [82]:
def parse_text_file(file_path):
    result_dict = {}

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            # 줄의 앞뒤 공백 제거
            line = line.strip()

            # C?? 형식의 코드와 나머지 문자열 분리
            if line:
                parts = line.split(' ', 1)  # 첫 번째 공백으로 나눔
                if len(parts) == 2:
                    key = parts[0]
                    values = parts[1].split(',')  # 쉼표와 공백으로 구분하여 리스트 생성
                    result_dict[key] = values

    return result_dict

In [305]:
def count_matches(keyword_list, reference_list):
    return sum(1 for keyword in keyword_list if keyword in reference_list)

In [210]:
def count_keyword_matches(keywords, ksic_key):
    result = {} # empty dict
    
    for ksic in ksic_key.items():
        intersect = keywords.intersection(ksic[1])
        if len(intersect) > 0 :
            result[ksic[0]] = intersect
            
    return result

In [188]:
def get_max_ksic(match_set):
    if len(match_set) == 0 :
        return {}
    
    results = {key for key, value in match_set.items() if len(value) >= 3}
    max_set_size = max(len(value) for value in match_set.values())
    results.update({key for key, value in match_set.items() if len(value) == max_set_size})
    
    return results

In [321]:
directory_path = '/home/osung/data/korean/modu/json'
df = pd.read_csv(directory_path+'/news_T_two_keywords.tsv', sep='\t')

In [322]:
df

,id,topic,text,keywords_textrank,keywords_tfid,keywords,match_count
0,NWRW1900000006.12652,기술,"스마트폰 충전… 이젠 오줌에 맡겨볼까 소변, 에너지로 부활… 휴대폰 충전·수소車 연...","['소변', '전지', '연료', '탄소', '촉매', '전자', '전기', '구진...","['소변', '연료', '전지', '탄소', '촉매', '전자', '요소', '스마...","{'전자', '구진', '연료', '소변', '전기', '요소', '촉매', '탄소...",9
1,NIRW1900000024.4104,기술,"삼성전자 5대 생활가전, 스마트 가전 시대 주도 삼성전자가 냉장고·에어컨·세탁기·오...","['가전', '제품', '스마트', '전자', '에어컨', '스마트폰', '청소',...","['제품', '가전', '스마트', '전자', '삼성', '스마트폰', '에어컨',...","{'전자', '가전', '기능', '냉장고', '온도', '스마트', '삼성', '...",9
2,NWRW1900000006.12584,기술,스마트폰 충전… 이젠 오줌에 맡겨볼까 머지않아 스마트폰의 배터리가 바닥나면 화장실로...,"['소변', '전지', '연료', '탄소', '촉매', '전자', '전기', '구진...","['소변', '전지', '연료', '촉매', '탄소', '전자', '전기', '요소...","{'전자', '구진', '연료', '소변', '전기', '요소', '촉매', '탄소...",8
3,NWRW1900000054.11539,기술,‘아이폰5’ 삼성-애플 대결구도 바꾸나 12일(현지시각) 미국 뉴욕에서 공개될 것으...,"['부품', '전자', '애플', '아이폰', '스마트폰', '삼성', '갤럭시',...","['부품', '전자', '애플', '아이폰', '갤럭시', '삼성', '스마트폰',...","{'전자', '경쟁력', '부품', '반도체', '삼성', '메모리', '애플', ...",8
4,NPRW1900000058.23991,기술,삼성 스마트폰으로 벤츠 문 여닫아…`IoT 공룡` 출현 ◆ IFA 2016 ◆ 시장...,"['스마트', '냉장고', '센서', '전자', '스마트폰', '허브', '가전',...","['스마트', '냉장고', '전자', '센서', '스마트폰', '서비스', '벤츠'...","{'통해', '전자', '가전', '기능', '냉장고', '차량', '벤츠', '스...",8
...,...,...,...,...,...,...,...
251509,NIRW2100000003.28753,기술,"미 하원 민주당 ""대형 기술기업 독점·지배력 폐단...규제·해체 필요"" 미국 하원 ...","['독점', '민주당', '구글', '기술', '페이스북', '대형', '아마존',...","['독점', '페이스북', '아마존', '민주당', '하원', '지배', '구글',...","{'기술', '페이스북', '아마존', '미국', '위원회', '하원', '대형',...",1
251510,NIRW2100000003.28768,기술,"네이버, AI 기술 경쟁력 강화...'네이버 AI랩' 신설 한국-일본-유럽-베트남에...","['연구', '네이버', '기술', '더욱', '장기', '조직', '확대', '선...","['네이버', '연구', '기술', '더욱', '확대', '조직', '장기', '선...","{'기술', '선행', '통해', '네이버', '장기', '조직', '연구소', '...",1
251511,NPRW1900000062.5839,기술,KT ´퉁´ 으로 LGT ´yo´ 에 맞불…요금 경쟁 ´격화´ KT는 지난 4월 출...,"['가족', '무료', '상품', '금제', '통화량', '요금', '구성원', '...","['가족', '상품', '금제', '무료', '요금', '통화량', '구성원', '...","{'통화량', '금제', '가족', '구성원', '가구', '사용', '가입', '...",1
251512,NPRW1900000063.3051,기술,"kt-케이티스, 광고사업 상호협력 협정 체결 KT와 케이티스는 광고사업 상호협력 협...","['서비스', '사업', '광고', '티스', '케이', '협정', '안내', '번...","['서비스', '광고', '사업', '케이', '티스', '번호', '협정', '협...","{'케이', '안내', '협력', '번호', '사업', '협정', '티스', '서비...",1


In [323]:
df['keywords_textrank'] = df['keywords_textrank'].apply(string_to_list)
df['keywords_tfid'] = df['keywords_tfid'].apply(string_to_list)

In [295]:
df

,id,topic,text,keywords_textrank,keywords_tfid,keywords,match_count
0,NWRW1900000006.12652,기술,"스마트폰 충전… 이젠 오줌에 맡겨볼까 소변, 에너지로 부활… 휴대폰 충전·수소車 연...","[소변, 전지, 연료, 탄소, 촉매, 전자, 전기, 구진, 수소, 질소]","[소변, 연료, 전지, 탄소, 촉매, 전자, 요소, 스마트폰, 수소, 전기]","{'전자', '구진', '연료', '소변', '전기', '요소', '촉매', '탄소...",9
1,NIRW1900000024.4104,기술,"삼성전자 5대 생활가전, 스마트 가전 시대 주도 삼성전자가 냉장고·에어컨·세탁기·오...","[가전, 제품, 스마트, 전자, 에어컨, 스마트폰, 청소, 생활, 로봇, 냉장고]","[제품, 가전, 스마트, 전자, 삼성, 스마트폰, 에어컨, 기능, 온도, 오븐]","{'전자', '가전', '기능', '냉장고', '온도', '스마트', '삼성', '...",9
2,NWRW1900000006.12584,기술,스마트폰 충전… 이젠 오줌에 맡겨볼까 머지않아 스마트폰의 배터리가 바닥나면 화장실로...,"[소변, 전지, 연료, 탄소, 촉매, 전자, 전기, 구진, 질소, 요소]","[소변, 전지, 연료, 촉매, 탄소, 전자, 전기, 요소, 교수, 스마트폰]","{'전자', '구진', '연료', '소변', '전기', '요소', '촉매', '탄소...",8
3,NWRW1900000054.11539,기술,‘아이폰5’ 삼성-애플 대결구도 바꾸나 12일(현지시각) 미국 뉴욕에서 공개될 것으...,"[부품, 전자, 애플, 아이폰, 스마트폰, 삼성, 갤럭시, 반도체, 더욱, 메모리]","[부품, 전자, 애플, 아이폰, 갤럭시, 삼성, 스마트폰, 메모리, 반도체, 경쟁력]","{'전자', '경쟁력', '부품', '반도체', '삼성', '메모리', '애플', ...",8
4,NPRW1900000058.23991,기술,삼성 스마트폰으로 벤츠 문 여닫아…`IoT 공룡` 출현 ◆ IFA 2016 ◆ 시장...,"[스마트, 냉장고, 센서, 전자, 스마트폰, 허브, 가전, 서비스, 통해, 차량]","[스마트, 냉장고, 전자, 센서, 스마트폰, 서비스, 벤츠, 기능, 허브, 차량]","{'통해', '전자', '가전', '기능', '냉장고', '차량', '벤츠', '스...",8
...,...,...,...,...,...,...,...
251509,NIRW2100000003.28753,기술,"미 하원 민주당 ""대형 기술기업 독점·지배력 폐단...규제·해체 필요"" 미국 하원 ...","[독점, 민주당, 구글, 기술, 페이스북, 대형, 아마존, 기업, 위원회, 애플]","[독점, 페이스북, 아마존, 민주당, 하원, 지배, 구글, 보고서, 기업, 미국]","{'기술', '페이스북', '아마존', '미국', '위원회', '하원', '대형',...",1
251510,NIRW2100000003.28768,기술,"네이버, AI 기술 경쟁력 강화...'네이버 AI랩' 신설 한국-일본-유럽-베트남에...","[연구, 네이버, 기술, 더욱, 장기, 조직, 확대, 선행, 통해, 연구소]","[네이버, 연구, 기술, 더욱, 확대, 조직, 장기, 선행, 통해, 연구소]","{'기술', '선행', '통해', '네이버', '장기', '조직', '연구소', '...",1
251511,NPRW1900000062.5839,기술,KT ´퉁´ 으로 LGT ´yo´ 에 맞불…요금 경쟁 ´격화´ KT는 지난 4월 출...,"[가족, 무료, 상품, 금제, 통화량, 요금, 구성원, 가입, 가구, 사용]","[가족, 상품, 금제, 무료, 요금, 통화량, 구성원, 가입, 가구, 사용]","{'통화량', '금제', '가족', '구성원', '가구', '사용', '가입', '...",1
251512,NPRW1900000063.3051,기술,"kt-케이티스, 광고사업 상호협력 협정 체결 KT와 케이티스는 광고사업 상호협력 협...","[서비스, 사업, 광고, 티스, 케이, 협정, 안내, 번호, 협력, 체결]","[서비스, 광고, 사업, 케이, 티스, 번호, 협정, 협력, 안내, 연결]","{'케이', '안내', '협력', '번호', '사업', '협정', '티스', '서비...",1


In [324]:
keywords_set = []

for row in df.iterrows():
    keywords_set.append(set(row[1]['keywords_textrank'] + row[1]['keywords_tfid']))

In [325]:
df['keywords'] = keywords_set

In [326]:
df[:10]

,id,topic,text,keywords_textrank,keywords_tfid,keywords,match_count
0,NWRW1900000006.12652,기술,"스마트폰 충전… 이젠 오줌에 맡겨볼까 소변, 에너지로 부활… 휴대폰 충전·수소車 연...","[소변, 전지, 연료, 탄소, 촉매, 전자, 전기, 구진, 수소, 질소]","[소변, 연료, 전지, 탄소, 촉매, 전자, 요소, 스마트폰, 수소, 전기]","{전자, 구진, 연료, 소변, 전기, 요소, 촉매, 탄소, 질소, 전지, 수소, 스...",9
1,NIRW1900000024.4104,기술,"삼성전자 5대 생활가전, 스마트 가전 시대 주도 삼성전자가 냉장고·에어컨·세탁기·오...","[가전, 제품, 스마트, 전자, 에어컨, 스마트폰, 청소, 생활, 로봇, 냉장고]","[제품, 가전, 스마트, 전자, 삼성, 스마트폰, 에어컨, 기능, 온도, 오븐]","{전자, 가전, 기능, 냉장고, 온도, 스마트, 삼성, 생활, 청소, 제품, 로봇,...",9
2,NWRW1900000006.12584,기술,스마트폰 충전… 이젠 오줌에 맡겨볼까 머지않아 스마트폰의 배터리가 바닥나면 화장실로...,"[소변, 전지, 연료, 탄소, 촉매, 전자, 전기, 구진, 질소, 요소]","[소변, 전지, 연료, 촉매, 탄소, 전자, 전기, 요소, 교수, 스마트폰]","{전자, 구진, 연료, 소변, 전기, 요소, 촉매, 탄소, 질소, 전지, 교수, 스...",8
3,NWRW1900000054.11539,기술,‘아이폰5’ 삼성-애플 대결구도 바꾸나 12일(현지시각) 미국 뉴욕에서 공개될 것으...,"[부품, 전자, 애플, 아이폰, 스마트폰, 삼성, 갤럭시, 반도체, 더욱, 메모리]","[부품, 전자, 애플, 아이폰, 갤럭시, 삼성, 스마트폰, 메모리, 반도체, 경쟁력]","{전자, 경쟁력, 부품, 반도체, 삼성, 메모리, 애플, 갤럭시, 더욱, 스마트폰,...",8
4,NPRW1900000058.23991,기술,삼성 스마트폰으로 벤츠 문 여닫아…`IoT 공룡` 출현 ◆ IFA 2016 ◆ 시장...,"[스마트, 냉장고, 센서, 전자, 스마트폰, 허브, 가전, 서비스, 통해, 차량]","[스마트, 냉장고, 전자, 센서, 스마트폰, 서비스, 벤츠, 기능, 허브, 차량]","{통해, 전자, 가전, 기능, 냉장고, 차량, 벤츠, 스마트, 허브, 서비스, 센서...",8
5,NPRW1900000006.3078,기술,"""연말 성수기 공략""…삼성·애플, 신제품 빅매치 세계 스마트폰 시장에 전운이 감돌고...","[전자, 애플, 시장, 스마트폰, 스마트, 갤럭시, 워치, 노트, 인치, 아이폰]","[전자, 애플, 시장, 스마트, 갤럭시, 워치, 스마트폰, 노트, 인치, 기어]","{전자, 워치, 스마트, 인치, 시장, 애플, 갤럭시, 노트, 기어, 스마트폰, 아이폰}",8
6,NLRW1900000119.443,기술,"모바일 월드 콩그레스 2015…인터넷, 3명부터 찜했다! 시계·지갑·자동차 세계 최...","[모바일, 결제, 워치, 자동차, 지갑, 서비스, 시계, 스마트폰, 차량, 스마트워치]","[모바일, 결제, 자동차, 스마트, 스마트폰, 워치, 차량, 페이, 지갑, 서비스]","{시계, 페이, 모바일, 결제, 워치, 차량, 스마트, 자동차, 서비스, 스마트폰,...",8
7,NPRW1900000004.3776,기술,"글로넷시스템즈, TV·노트북 등 고화질 LCD화면 구현 국내 중소기업이 영상 전송 ...","[기술, 영상, 전송, 반도체, 디지털, 신호, 노트북, 개발, 모니터, 고화질]","[기술, 영상, 반도체, 전송, 디스플레이, 개발, 노트북, 신호, 디지털, 패널]","{기술, 노트북, 개발, 고화질, 반도체, 디스플레이, 영상, 패널, 신호, 모니터...",8
8,NPRW1900000055.24070,기술,"""스마트폰이 IT 중심…무조건 올라타야"" 삼성전기 사장의 로켓론 ◆ 갤럭시 쏠림의 ...","[삼성, 전자, 갤럭시, 스마트폰, 로켓, 부품, 모바일, 디스플레이, 휴대폰, 계열]","[삼성, 갤럭시, 스마트폰, 부품, 전자, 계열, 모바일, 디스플레이, 사업, 시장]","{전자, 모바일, 로켓, 부품, 휴대폰, 디스플레이, 삼성, 시장, 갤럭시, 사업,...",8
9,NIRW1900000018.2398,기술,아이디어 상품 7종 `아이디어 쇼케이스`는 출시 준비 중 혹은 이미 시중에 선보인 ...,"[아이폰, 사용, 충전기, 아이, 빙삭기, 애플, 제품, 케이블, 마우스, 메모리]","[아이폰, 사용, 제품, 애플, 아이, 충전기, 바이오, 배터리, 빙삭기, 마우스]","{케이블, 충전기, 아이, 메모리, 빙삭기, 마우스, 사용, 애플, 제품, 바이오,...",8


In [327]:
file_path = '/home/osung/data/korean/KSIC_keyword.txt'
KSIC_S_keywords = parse_text_file(file_path)

In [328]:
KSIC_S_keywords

{'c101': ['도축장',
  '도축',
  '축산물',
  '도축업',
  '도축',
  '육류',
  '한우',
  '냉동',
  '토종닭',
  '축산',
  '마리',
  '사육',
  '고래고기',
  '햄',
  '소시지',
  '선지',
  '소고기',
  '돼지고기',
  '양고기',
  '오리고기',
  '닭고기',
  '훈제',
  '닭가슴살',
  '닭발',
  '닭갈비',
  '닭꼬치',
  '양꼬치',
  '돈까스',
  '돈가스',
  '족발',
  '갈비',
  '베이컨',
  '편육'],
 'c102': ['수산물',
  '양식',
  '수산',
  '광어',
  '항만',
  '해조',
  '어란',
  '어육',
  '캐비어',
  '어묵',
  '젓갈',
  '생선통조림',
  '생선페이스트',
  '건어물',
  '오징어',
  '우럭',
  '도미',
  '도다리',
  '가자미',
  '갈치',
  '고등어',
  '멍게',
  '해삼',
  '굴',
  '석화',
  '조개',
  '가리비',
  '키조개',
  '참돔',
  '돌돔',
  '줄돔',
  '방어',
  '쥐포',
  '먹태',
  '황태',
  '게장',
  '맛살',
  '생선가스',
  '생선까스',
  '액젓',
  '북어채',
  '황태채',
  '미역',
  '김부각',
  '구운김',
  '조미김'],
 'c103': ['농업',
  '농가',
  '농산물',
  '농협',
  '식품',
  '양파',
  '마늘',
  '과일',
  '채소주스',
  '과일주스',
  '과채주스',
  '알로에',
  '과실주스',
  '딸기잼',
  '사과잼',
  '포도잼',
  '과일페이스트',
  '과일통조림',
  '과일샐러드',
  '김치',
  '단무지',
  '건채소',
  '건과일',
  '장아찌',
  '무쌈',
  '쌈무',
  '오이지',
  '피클',
  '감귤주스',
  '녹즙',
  '매실액',
  '배즙',
  '사과즙',
 

In [329]:
KSIC_key = None

In [330]:
combined_list = []

for key in KSIC_S_keywords :
    combined_list.extend(KSIC_S_keywords[key])

In [331]:
# 각 행에 대해 일치하는 문자열의 개수를 세고 새로운 열에 저장
df['match_count'] = df['keywords'].apply(lambda x: count_matches(x, combined_list))

In [332]:
df = df.sort_values(by='match_count', ascending=False)

In [359]:
df[:300]

,id,topic,text,keywords_textrank,keywords_tfid,keywords,match_count,match_S_list,match_M_list,max_S_ksic,max_M_ksic
0,NWRW1900000006.12652,기술,"스마트폰 충전… 이젠 오줌에 맡겨볼까 소변, 에너지로 부활… 휴대폰 충전·수소車 연...","[소변, 전지, 연료, 탄소, 촉매, 전자, 전기, 구진, 수소, 질소]","[소변, 연료, 전지, 탄소, 촉매, 전자, 요소, 스마트폰, 수소, 전기]","{전자, 구진, 연료, 소변, 전기, 요소, 촉매, 탄소, 질소, 전지, 수소, 스...",9,"{'c191': {'연료'}, 'c192': {'연료'}, 'c201': {'수소'...","{'c19': {'연료'}, 'c20': {'수소', '탄소', '요소', '질소'...","{c261, c201}","{c20, c26}"
1,NIRW1900000024.4104,기술,"삼성전자 5대 생활가전, 스마트 가전 시대 주도 삼성전자가 냉장고·에어컨·세탁기·오...","[가전, 제품, 스마트, 전자, 에어컨, 스마트폰, 청소, 생활, 로봇, 냉장고]","[제품, 가전, 스마트, 전자, 삼성, 스마트폰, 에어컨, 기능, 온도, 오븐]","{전자, 가전, 기능, 냉장고, 온도, 스마트, 삼성, 생활, 청소, 제품, 로봇,...",9,"{'c261': {'스마트', '전자', '스마트폰'}, 'c262': {'스마트'...","{'c26': {'스마트', '전자', '스마트폰'}, 'c27': {'온도'}, ...","{c285, c261, c281}","{c28, c26}"
10,NWRW1900000034.24966,기술,"LG, 최첨단 디스플레이 ‘G폰’ 다음달 출시 LG전자가 삼성전자와 애플 등에 대항...","[디스플레이, 기술, 제품, 화면, 유리, 다음, 터치, 커버, 센서, 부품]","[디스플레이, 기술, 전자, 제품, 터치, 다음, 화면, 스마트폰, 센서, 배터리]","{기술, 전자, 부품, 유리, 디스플레이, 커버, 화면, 다음, 터치, 제품, 센서...",8,"{'c171': {'커버'}, 'c179': {'커버'}, 'c231': {'유리'...","{'c17': {'커버'}, 'c23': {'유리'}, 'c26': {'전자', '...","{c262, c261}",{c26}
16,NLRW1900000062.4720,기술,베일 벗은 'LG G7 씽큐'… 색상 4가지·가격 90만원 안팎 'AI·카메라·스피...,"[탑재, 스피커, 스마트폰, 디스플레이, 기능, 전자, 카메라, 출시, 아이폰, 강화]","[탑재, 전자, 스마트폰, 디스플레이, 카메라, 기능, 스피커, 아이폰, 강화, 오디오]","{전자, 기능, 강화, 출시, 오디오, 디스플레이, 카메라, 탑재, 스마트폰, 스피...",8,"{'c181': {'오디오'}, 'c182': {'오디오'}, 'c231': {'강...","{'c18': {'오디오'}, 'c23': {'강화'}, 'c26': {'전자', ...","{c261, c264}",{c26}
15,NPRW1900000064.2940,기술,"삼성전자, ""5대 생활가전, 스마트폰으로 통한다"" 삼성전자는 냉장고·에어컨·세탁기·...","[제품, 가전, 스마트, 전자, 스마트폰, 냉장고, 에어컨, 생활, 기능, 청소]","[제품, 스마트, 가전, 전자, 스마트폰, 에어컨, 삼성, 기능, 냉장고, 오븐]","{전자, 가전, 기능, 냉장고, 스마트, 삼성, 생활, 청소, 제품, 오븐, 스마트...",8,"{'c261': {'스마트', '전자', '스마트폰'}, 'c262': {'스마트'...","{'c26': {'스마트', '전자', '스마트폰'}, 'c28': {'에어컨', ...","{c285, c261, c281}","{c28, c26}"
...,...,...,...,...,...,...,...,...,...,...,...
566,NWRW1900000009.13188,기술,"평면에서 입체로 저장공간 확대… 낸드플래시, 72층까지 쌓았다 SK하이닉스가 참여한...","[플래시, 낸드, 저장, 정보, 전자, 메모리, 데이터, 반도체, 스마트폰, 제품]","[플래시, 저장, 낸드, 정보, 전자, 메모리, 데이터, 반도체, 제품, 처리]","{정보, 전자, 처리, 반도체, 저장, 제품, 낸드, 데이터, 플래시, 스마트폰, ...",6,"{'c251': {'저장'}, 'c261': {'메모리', '전자', '스마트폰',...","{'c25': {'저장'}, 'c26': {'전자', '스마트폰', '메모리', '...",{c261},{c26}
567,NPRW2000000004.53252,기술,"'모바일로 최대토크·응답성까지 조절', 현대·기아차 세계 최초 개발 차량과 모바일 ...","[기술, 차량, 공유, 기아차, 성능, 전기차, 개발, 설정, 스마트폰, 모빌리티]","[차량, 기술, 공유, 전기차, 기아차, 성능, 설정, 스마트폰, 자동차, 개발]","{기술, 차량, 모빌리티, 기아차, 스마트폰, 성능, 자동차, 전기차, 개발, 설정...",6,"{'c251': {'자동차'}, 'c259': {'자동차'}, 'c261': {'스...","{'c25': {'자동차'}, 'c26': {'스마트폰'}, 'c30': {'자동차...",{c301},{c30}
568,NIRW2100000003.26394,기술,"삼성·SK·LG, 美 '화웨이 제재' 직격타…관련업계 지형 변화 올까 중국 화웨이를...","[화웨이, 반도체, 미국, 제재, 전자, 공급, 디스플레이, 업체, 메모리, 스마트폰]","[화웨이, 반도체, 미국, 공급, 제재, 메모리, 디스플레이, 업체, 전자, 기업]","{제재, 전자, 업체, 공급, 화웨이, 반도체, 디스플레이, 기업, 스마트폰, 메모...",6,"{'c161': {'제재'}, 'c261': {'전자', '반도체', '디스플레이'...","{'c16': {'제재'}, 'c26': {'전자', '반도체', '디스플레이', ...",{c261},{c26}
569,NPRW1900000055.15306,기술,삼성·LG `휘는 OLED` 격돌…화면 안깨지는 폰 연내 나올듯 세계 시장의 절반을...,"[디스플레이, 레드, 플라스틱, 스마트폰, 패널, 적용, 출시, 전자, 스마트, 화면]","[디스플레이, 플라스틱, 레드, 스마트폰, 패널, 스마트, 출시, 제품, 적용, 삼성]","{전자, 플라스틱, 레드, 출시, 스마트, 디스플레이, 삼성, 화면, 패널, 적용,...",6,"{'c201': {'플라스틱'}, 'c202': {'플라스틱'}, 'c221': {...","{'c20': {'플라스틱'}, 'c22': {'플라스틱'}, 'c26': {'전자...","{c262, c261}",{c26}


In [335]:
len(df)

251514

In [96]:
df.to_csv(directory_path+'/news_T_two_keywords.tsv', sep='\t', index=False)

In [316]:
KSIC_S_keywords

{'c101': ['도축장',
  '도축',
  '축산물',
  '도축업',
  '도축',
  '육류',
  '한우',
  '냉동',
  '토종닭',
  '축산',
  '마리',
  '사육',
  '고래고기',
  '햄',
  '소시지',
  '선지',
  '소고기',
  '돼지고기',
  '양고기',
  '오리고기',
  '닭고기',
  '훈제',
  '닭가슴살',
  '닭발',
  '닭갈비',
  '닭꼬치',
  '양꼬치',
  '돈까스',
  '돈가스',
  '족발',
  '갈비',
  '베이컨',
  '편육'],
 'c102': ['수산물',
  '양식',
  '수산',
  '광어',
  '항만',
  '해조',
  '어란',
  '어육',
  '캐비어',
  '어묵',
  '젓갈',
  '생선통조림',
  '생선페이스트',
  '건어물',
  '오징어',
  '우럭',
  '도미',
  '도다리',
  '가자미',
  '갈치',
  '고등어',
  '멍게',
  '해삼',
  '굴',
  '석화',
  '조개',
  '가리비',
  '키조개',
  '참돔',
  '돌돔',
  '줄돔',
  '방어',
  '쥐포',
  '먹태',
  '황태',
  '게장',
  '맛살',
  '생선가스',
  '생선까스',
  '액젓',
  '북어채',
  '황태채',
  '미역',
  '김부각',
  '구운김',
  '조미김'],
 'c103': ['농업',
  '농가',
  '농산물',
  '농협',
  '식품',
  '양파',
  '마늘',
  '과일',
  '채소주스',
  '과일주스',
  '과채주스',
  '알로에',
  '과실주스',
  '딸기잼',
  '사과잼',
  '포도잼',
  '과일페이스트',
  '과일통조림',
  '과일샐러드',
  '김치',
  '단무지',
  '건채소',
  '건과일',
  '장아찌',
  '무쌈',
  '쌈무',
  '오이지',
  '피클',
  '감귤주스',
  '녹즙',
  '매실액',
  '배즙',
  '사과즙',
 

In [336]:
# KSIC M(edium) Keywords 생성 
KSIC_M_keywords = {}

for key, value in KSIC_S_keywords.items():
    # key의 마지막 글자 제거
    new_key = key[:-1]
    
    # 기존에 존재하는 key라면 value 병합
    if new_key in KSIC_M_keywords:
        KSIC_M_keywords[new_key].extend(value)
    else:
        KSIC_M_keywords[new_key] = value

# 병합된 list에서 중복 제거
for key in merged_data:
    KSIC_M_keywords[key] = list(set(KSIC_M_keywords[key]))

In [337]:
KSIC_M_keywords

{'c10': ['올리브유',
  '수산',
  '수산물',
  '피자',
  '강아지',
  '과실주스',
  '생선페이스트',
  '낙농',
  '도시락',
  '줄돔',
  '고양이',
  '홍삼차',
  '이눌린',
  'pet',
  '농축유',
  '유당',
  '농산물',
  '단무지',
  '고추가루',
  '수프',
  '두유',
  '떡고물',
  '앙금',
  '과일',
  '현미',
  '계피가루',
  '정제유',
  '황태',
  '떡',
  '유부',
  '치즈',
  '밀가루',
  '사료',
  '청국장',
  '양꼬치',
  '김치',
  '조미김',
  '젓갈',
  '먹태',
  '오징어',
  '선지',
  '빙수',
  '춘장',
  '인삼즙',
  '과일샐러드',
  '콩가루',
  '감말랭이',
  '파이',
  '마요네즈',
  '맛소금',
  '참돔',
  '쇼트닝',
  '유제품',
  '이스트',
  '생선까스',
  '알로에',
  '건강원',
  '인삼차',
  '구운김',
  '우럭',
  '귀리',
  '건채소',
  '물엿',
  '돼지고기',
  '케이크',
  '쌈무',
  '식초',
  '고래고기',
  '두부',
  '햄스터',
  '과자',
  '도축장',
  '족발',
  '생선통조림',
  '참기름',
  '곡류',
  '고등어',
  '베이커리',
  '굴',
  '과채주스',
  '가자미',
  '피클',
  '개껌',
  '초콜릿',
  '정미소',
  '농가',
  '옥수수기름',
  '동물',
  '갈치',
  '석화',
  '사과잼',
  '옥수수가루',
  '게장',
  '베이킹파우더',
  '닭꼬치',
  '황태채',
  '양식',
  '도다리',
  '건과일',
  '한우',
  '닭발',
  '과일주스',
  '피마자유',
  '반죽',
  '홍차',
  '김부각',
  '마늘',
  '육류',
  '항만',
  '메밀국수',
  '도축업',
  '고추장',
  '가리비'

In [338]:
match_S_list = []

for index, row in tqdm(df.iterrows()):
    match_S_list.append(count_keyword_matches(row['keywords'], KSIC_S_keywords))

251514it [01:48, 2322.58it/s]


In [339]:
len(match_S_list)

251514

In [340]:
match_M_list = []

for index, row in tqdm(df.iterrows()):
    #print(row['keywords'], row['match_count'])
    match_M_list.append(count_keyword_matches(row['keywords'], KSIC_M_keywords))

251514it [01:16, 3269.98it/s]


In [343]:
match_M_list[:3]

[{'c19': {'연료'},
  'c20': {'수소', '요소', '질소', '탄소'},
  'c23': {'탄소'},
  'c26': {'스마트폰', '전기', '전자'},
  'c28': {'전기', '전지'},
  'c30': {'전기'}},
 {'c26': {'스마트', '스마트폰', '전자'},
  'c27': {'온도'},
  'c28': {'가전', '냉장고', '에어컨', '청소'},
  'c29': {'오븐'},
  'c30': {'에어컨'}},
 {'c17': {'커버'},
  'c23': {'유리'},
  'c26': {'디스플레이', '센서', '스마트폰', '전자'},
  'c27': {'센서'},
  'c28': {'배터리'},
  'c30': {'부품'},
  'c31': {'부품'}}]

In [344]:
max_S_ksic = []

for match in tqdm(match_S_list) :
    max_S_ksic.append(get_max_ksic(match))

100%|████████████████████████████████| 251514/251514 [00:22<00:00, 10969.09it/s]


In [345]:
max_M_ksic = []

for match in tqdm(match_M_list) :
    max_M_ksic.append(get_max_ksic(match))

100%|████████████████████████████████| 251514/251514 [00:19<00:00, 13039.86it/s]


In [346]:
max_S_ksic

[{'c201', 'c261'},
 {'c261', 'c281', 'c285'},
 {'c261', 'c262'},
 {'c261', 'c264'},
 {'c261', 'c281', 'c285'},
 {'c151'},
 {'c271'},
 {'c261', 'c264'},
 {'c261', 'c264'},
 {'c281', 'c285'},
 {'c261', 'c281'},
 {'c261', 'c271'},
 {'c261'},
 {'c261', 'c264'},
 {'c261', 'c262'},
 {'c261', 'c264'},
 {'c201', 'c261'},
 {'c261', 'c264'},
 {'c261', 'c264'},
 {'c261', 'c264'},
 {'c261', 'c264'},
 {'c261', 'c262'},
 {'c261', 'c262', 'c264'},
 {'c261', 'c262'},
 {'c261', 'c262', 'c271', 'c281', 'c285'},
 {'c261', 'c264'},
 {'c281', 'c285'},
 {'c261', 'c264'},
 {'c261'},
 {'c211', 'c271'},
 {'c281', 'c285'},
 {'c261'},
 {'c261'},
 {'c171', 'c172', 'c211', 'c271', 'c311', 'c313'},
 {'c281'},
 {'c261', 'c271'},
 {'c211', 'c212', 'c281'},
 {'c261', 'c264', 'c271', 'c273'},
 {'c261', 'c264'},
 {'c261', 'c262'},
 {'c261', 'c281', 'c285'},
 {'c261'},
 {'c261'},
 {'c281', 'c285'},
 {'c201'},
 {'c261', 'c264', 'c271'},
 {'c261'},
 {'c261', 'c264'},
 {'c261'},
 {'c261', 'c264'},
 {'c261', 'c271', 'c273'},

In [347]:
max_M_ksic

[{'c20', 'c26'},
 {'c26', 'c28'},
 {'c26'},
 {'c26'},
 {'c26', 'c28'},
 {'c15'},
 {'c27'},
 {'c26'},
 {'c26'},
 {'c28'},
 {'c26', 'c28'},
 {'c26', 'c27'},
 {'c26'},
 {'c26'},
 {'c26'},
 {'c26'},
 {'c20', 'c26'},
 {'c26'},
 {'c26'},
 {'c26'},
 {'c26'},
 {'c26'},
 {'c26'},
 {'c26'},
 {'c26', 'c27', 'c28'},
 {'c26'},
 {'c28'},
 {'c26'},
 {'c26'},
 {'c21', 'c27'},
 {'c28'},
 {'c26'},
 {'c26'},
 {'c17', 'c21', 'c27', 'c31'},
 {'c28'},
 {'c26', 'c27'},
 {'c21', 'c28'},
 {'c26', 'c27'},
 {'c26'},
 {'c26'},
 {'c26', 'c28'},
 {'c26'},
 {'c26'},
 {'c28'},
 {'c20'},
 {'c26', 'c27'},
 {'c26'},
 {'c26'},
 {'c26'},
 {'c26'},
 {'c26', 'c27'},
 {'c13', 'c28'},
 {'c26'},
 {'c26'},
 {'c26'},
 {'c26'},
 {'c27'},
 {'c31'},
 {'c15', 'c26', 'c27'},
 {'c27'},
 {'c26', 'c28'},
 {'c26'},
 {'c26'},
 {'c26'},
 {'c28'},
 {'c26'},
 {'c26'},
 {'c27'},
 {'c31'},
 {'c26'},
 {'c26'},
 {'c27', 'c31'},
 {'c26', 'c27'},
 {'c26', 'c27'},
 {'c26', 'c27'},
 {'c26'},
 {'c26', 'c28', 'c30'},
 {'c26', 'c27'},
 {'c27'},
 {'c26'

In [351]:
df['match_S_list'] = match_S_list

In [352]:
df['match_M_list'] = match_M_list

In [353]:
df['max_S_ksic'] = max_S_ksic

In [354]:
df['max_M_ksic'] = max_M_ksic

In [356]:
df.to_csv(directory_path+'/news_T_KSIC_keywords.tsv', sep='\t', index=False)

In [355]:
df.columns

Index(['id', 'topic', 'text', 'keywords_textrank', 'keywords_tfid', 'keywords',
       'match_count', 'match_S_list', 'match_M_list', 'max_S_ksic',
       'max_M_ksic'],
      dtype='object')

In [360]:
max_count = 0
max_idx = 0

for index, ksic in enumerate(max_M_ksic[:1000]) :
    if max_count < len(ksic):
        max_count = len(ksic)
        max_idx = index

In [362]:
max_idx

890

In [363]:
max_M_ksic[890]

{'c13', 'c14', 'c19', 'c20', 'c23', 'c27', 'c28', 'c31', 'c32'}

In [364]:
df.iloc[890]

id                                                NPRW1900000068.32462
topic                                                               기술
text                 린나이, 1~2인 가구 맞춤형 빨래건조기 '해밀' 출시 린나이코리아는 지속적인 증가...
keywords_textrank           [빨래, 사용, 건조기, 해밀, 옷감, 건조, 가스, 출시, 신제품, 의류]
keywords_tfid               [빨래, 가구, 건조, 옷감, 사용, 건조기, 나이, 온도, 바람, 사이즈]
keywords             {온도, 신제품, 출시, 건조기, 해밀, 옷감, 가구, 건조, 사용, 가스, 사이즈...
match_count                                                          6
match_S_list         {'c131': {'의류'}, 'c134': {'의류'}, 'c141': {'의류'...
match_M_list         {'c13': {'의류'}, 'c14': {'의류'}, 'c19': {'가스'}, ...
max_S_ksic           {c191, c231, c285, c311, c272, c131, c192, c20...
max_M_ksic               {c14, c20, c27, c19, c31, c28, c23, c13, c32}
Name: 366, dtype: object

In [365]:
row = df.iloc[890]

In [367]:
row.match_M_list

{'c13': {'의류'},
 'c14': {'의류'},
 'c19': {'가스'},
 'c20': {'가스'},
 'c23': {'가구'},
 'c27': {'온도'},
 'c28': {'건조기'},
 'c31': {'건조'},
 'c32': {'가구'}}

In [368]:
row.text

'린나이, 1~2인 가구 맞춤형 빨래건조기 \'해밀\' 출시 린나이코리아는 지속적인 증가세를 보이고 있는 1~2인 가구를 겨냥한 콤팩트한 사이즈의 빨래건조기 신제품 \'해밀\'(모델명 RD-41S)을 출시했다고 18일 밝혔다. 이번 신제품은 기존 모델인 RD-61S 대비 사이즈를 30% 축소했으며 무게는 27.4kg다. 옷감에 따라 표준, 민감, 소프트 등 총 5가지 코스를 선택할 수 있으며 4kg까지 건조가 가능하다. 8단계 비례제어방식을 적용해 열풍의 온도를 90~100℃로 일정하게 유지해 의류 손상이 없고 효율이 높아 가스비를 절감할 수 있는 것이 특징이다. 90℃의 강력한 열풍으로 옷감 내에 남아있을 수 있는 세균이나 진드기를 없애주며, 옷감 사이사이에 바람을 넣어 옷감의 구김을 막고 한 올 한 올 부드럽게 말려준다. 또 전자동 시스템(Auto Move/Stop System)을 사용, 건조가 끝나면 종료음과 함께 자동적으로 멈춰 연료의 낭비를 줄여준다. 빨래건조량 3kg 기준 1회 사용 가스비는 187원(실사용 표준모드 의류 3kg 건조기준. 전기료 포함)으로, 비용 부담을 덜었다. 린나이 의류건조기 특유의 빨래온도검지센서를 통해 빨래 온도를 직접 측정할 수 있고 과열, 문열림 방지장치 등 총 8개의 안전장치를 탑재하고 있어 효율성과 안전성이 뛰어나다. 국내 최초로 급기필터를 장착해 외부에서 들어오는 공기로부터 유입되는 먼지도 원천 차단했다. 린나이코리아 관계자는 "1~2인 가구가 전체 가구의 50%를 차지할 정도로 소비의 주체로 떠오르면서 소용량 가전 시장이 가파르게 성장하고 있다"며 "소형 아파트와 오피스텔을 중심으로 바람을 일으킬 것으로 기대된다"고 말했다. 이번에 만들어진 린나이 빨래건조기의 브랜드네임인 해밀은 비가 온 뒤 맑게 개인 하늘이란 의미의 한글 표현으로 비 온 뒤 깨끗하고 맑은 햇살에 널어 놓은 듯 상쾌하고 산뜻하게 건조시켜준다는 의미를 담고 있다.'

In [369]:
row.keywords

{'가구',
 '가스',
 '건조',
 '건조기',
 '나이',
 '바람',
 '빨래',
 '사용',
 '사이즈',
 '신제품',
 '온도',
 '옷감',
 '의류',
 '출시',
 '해밀'}

In [183]:
match_list[0]

{'c120': {'전자'},
 'c192': {'연료'},
 'c201': {'수소', '질소', '탄소'},
 'c203': {'요소', '질소'},
 'c239': {'탄소'},
 'c262': {'전기', '전자'},
 'c264': {'스마트폰'},
 'c282': {'전기', '전지'},
 'c289': {'전기'},
 'c301': {'전기'}}

In [358]:
df.iloc[275]

id                                                 NLRW2100000016.1919
topic                                                               기술
text                 최대 20회 세탁 가능한 마스크 나온다… KAIST 나노섬유 마스크 개발 KAIST...
keywords_textrank           [마스크, 섬유, 나노, 필터, 이상, 유지, 개발, 세척, 세탁, 에탄올]
keywords_tfid               [마스크, 나노, 섬유, 필터, 세탁, 개발, 이상, 유지, 세척, 에탄올]
keywords                    {섬유, 유지, 개발, 세척, 필터, 에탄올, 이상, 세탁, 나노, 마스크}
match_count                                                          6
match_S_list         {'c111': {'에탄올'}, 'c131': {'섬유', '마스크', '필터'},...
match_M_list         {'c11': {'에탄올'}, 'c13': {'섬유', '마스크', '필터'}, '...
max_S_ksic                                                      {c131}
max_M_ksic                                                       {c13}
Name: 596, dtype: object

In [191]:
len(max_ksic)

382513

In [192]:
len(max_ksic[0])

1

In [193]:
sorted_max_ksic = sorted(max_ksic, key=len, reverse=True)

In [194]:
sorted_max_ksic[:5]

[{'c103',
  'c112',
  'c172',
  'c179',
  'c203',
  'c204',
  'c213',
  'c221',
  'c243',
  'c284',
  'c285',
  'c311'},
 {'c131',
  'c134',
  'c192',
  'c201',
  'c221',
  'c239',
  'c261',
  'c262',
  'c271',
  'c272',
  'c273'},
 {'c181',
  'c204',
  'c242',
  'c259',
  'c262',
  'c271',
  'c282',
  'c283',
  'c289',
  'c292',
  'c301'},
 {'c131',
  'c134',
  'c171',
  'c204',
  'c221',
  'c239',
  'c242',
  'c262',
  'c282',
  'c289',
  'c301'},
 {'c134',
  'c143',
  'c203',
  'c213',
  'c262',
  'c282',
  'c284',
  'c285',
  'c289',
  'c301',
  'c311'}]

In [200]:
for s in sorted_max_ksic[:100] :
    print(len(s))

12
11
11
11
11
11
11
11
11
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9


In [209]:
for keys in KSIC_M_keywords.items():
    intersect = df.keywords[1].intersection(keys[1])
    if len(intersect) > 0 :
        print(keys[0], keys[1], intersect)

c25 ['롤링도어', '철골', '압형', '다리미', '현금함', '철모', '탄도', '증기발생기', '전차', '숟가락', '샤시', '나사', '핀셋', '쐐기', '공기총', '도장', '한화', '이동식사다리', '철문', '손톱깎이', '사다리', '핵반응기', '볼트', '면도날', '계단', '보일러', '칼날', '자물쇠', '거푸집', '경첩', '절단기', '디펜스', '면도기', '분쇄기', '미사일', '삽', '작두', '젓가락', '도로표지판', '날붙이', '카스터', '폭탄', '분말야금', '도끼', '스크린도어', '전지가위', '쇠스랑', '부유구조물', '원자로', '방위', '연탄집게', '탑', '너트', '저장', '도금', '셔터', '과도', '탱크', '격자판넬', '울타리', '열쇠', '금속스프링', '가스총', '톱', '금속분말', '명패', '자동차', '포탄', '전동차', '라디에이터', '방산', '낫', '우편함', '컨테이너', '스트롱박스', '집게', '곡괭이', '인두', '철망', '무기', '교량', '교통표지판', '상패', '식칼', '금형', '스패너', '국방', '교통안전표시판', '자주포', '스크루', '발사', '방열기', '금속단조', '조발기', '우체통', '가위', '철도궤도', '금고', '대검', '감사패', '렌치', '탄약', '피막'] {'발사'}
c31 ['발사체', '대우조선해양', '사이클', '모빌리티', '스쿠터', '현대로템', '탱크', '항공', '우주선', 'SRT', '활공기', '삼성중공업', '여객선', '대륙간탄도미사일', '보트', '손수레', '우마차', '차량', '궤도', '부품', '군함', '드론', '비행', '철도', '조선', '건조', '전차', '객차', '우주', '기관차', '이륜자동차', '항공기', '전동차', '유조선', '선박', '로켓', '쇼핑카트', '누리호', '로만시스', '전투용', '실버

In [211]:
match_M_list = []

for index, row in tqdm(df.iterrows()):
    #print(row['keywords'], row['match_count'])
    match_M_list.append(count_keyword_matches(row['keywords'], KSIC_M_keywords))

382513it [00:47, 8009.63it/s] 


In [212]:
match_M_list

[{'c12': {'전자'},
  'c19': {'연료'},
  'c20': {'수소', '요소', '질소', '탄소'},
  'c23': {'탄소'},
  'c26': {'스마트폰', '전기', '전자'},
  'c28': {'전기', '전지'},
  'c30': {'전기'}},
 {'c12': {'전자'},
  'c26': {'스마트', '스마트폰', '전자'},
  'c27': {'온도'},
  'c28': {'가전', '냉장고', '에어컨', '청소'},
  'c29': {'오븐'}},
 {'c12': {'전자'},
  'c19': {'연료'},
  'c20': {'요소', '질소', '탄소'},
  'c23': {'탄소'},
  'c26': {'스마트폰', '전기', '전자'},
  'c28': {'전기', '전지'},
  'c30': {'전기'}},
 {'c12': {'전자'},
  'c26': {'갤럭시', '메모리', '반도체', '스마트폰', '아이폰', '애플', '전자'},
  'c30': {'부품'},
  'c31': {'부품'}},
 {'c12': {'전자'},
  'c26': {'센서', '스마트', '스마트폰', '전자'},
  'c27': {'센서'},
  'c28': {'가전', '냉장고'},
  'c30': {'벤츠', '차량'},
  'c31': {'차량'}},
 {'c12': {'전자'},
  'c26': {'갤럭시', '스마트', '스마트폰', '아이폰', '애플', '전자'},
  'c27': {'워치'},
  'c29': {'기어'},
  'c30': {'기어'}},
 {'c15': {'시계', '지갑'},
  'c25': {'자동차'},
  'c26': {'모바일', '스마트', '스마트폰'},
  'c27': {'시계', '워치'},
  'c30': {'자동차', '차량'},
  'c31': {'차량'}},
 {'c26': {'노트북', '디스플레이', '모니터', '반도체', '패널'}, 'c27': {'디지털',

In [213]:
df.columns

Index(['id', 'topic', 'text', 'keywords_textrank', 'keywords_tfid', 'keywords',
       'match_count'],
      dtype='object')

In [214]:
len(df)

382513

In [215]:
len(match_M_list)

382513

In [216]:
df['match_M_list'] = match_M_list

In [217]:
df.columns

Index(['id', 'topic', 'text', 'keywords_textrank', 'keywords_tfid', 'keywords',
       'match_count', 'match_M_list'],
      dtype='object')

In [218]:
df.iloc[0]

id                                                NWRW1900000006.12652
topic                                                               기술
text                 스마트폰 충전… 이젠 오줌에 맡겨볼까 소변, 에너지로 부활… 휴대폰 충전·수소車 연...
keywords_textrank    ['소변', '전지', '연료', '탄소', '촉매', '전자', '전기', '구진...
keywords_tfid        ['소변', '연료', '전지', '탄소', '촉매', '전자', '요소', '스마...
keywords             {전자, 구진, 연료, 소변, 전기, 요소, 촉매, 탄소, 질소, 전지, 수소, 스...
match_count                                                          9
match_M_list         {'c12': {'전자'}, 'c19': {'연료'}, 'c20': {'수소', '...
Name: 297158, dtype: object

In [219]:
df.iloc[0].text

'스마트폰 충전… 이젠 오줌에 맡겨볼까 소변, 에너지로 부활… 휴대폰 충전·수소車 연료에 사용, 우주인은 食水로 재활용하기도 머지않아 스마트폰의 배터리가 바닥나면 화장실로 뛰어가는 모습을 볼 수 있을지 모른다. 사람의 소변을 연료 삼아 전기를 만드는 기술이 잇따라 등장하고 있기 때문. 장시간 우주여행을 대비해 소변을 재활용해 식수와 전기를 동시에 얻는 기술도 개발됐다. 국내에서는 소변으로 전지에 쓸 촉매를 만드는 데에도 성공했다. 소변이 에너지로의 화려한 변신을 시작한 것이다. ◇소변으로 작동한 삼성 스마트폰 70억의 인류가 하루에 배출하는 소변은 무려 105억L에 이른다. 올림픽 규격의 수영장 4200개를 채울 수 있는 엄청난 양이다. 영국 브리스톨대의 이오아니스 이에로풀로스(Ieropoulos) 교수는 소변이 전기의 혜택을 보지 못하는 저개발국가 10억의 인구에게 빛을 안겨줄 수 있다고 본다. 그가 주목한 것은 미생물 연료전지. 미생물이 폐수에 들어 있는 유기물을 분해할 때 나오는 전자를 이용하는 전지다. 전자의 흐름이 곧 전류이다. 소변은 98%가 물이고 2%는 탄소와 질소, 수소, 산소로 이뤄진 유기물인 요소(尿素)이다. 브리스톨대 연구진은 빌&멜린다 게이츠 재단의 지원을 받아 소변을 연료로 하는 미생물 연료전지를 개발했다. 지난해 연구진은 이 전지로 삼성 스마트폰을 충전하는 데 성공했다. 전류가 많이 필요한 통화는 어려웠지만, 문자를 보내거나 인터넷을 검색하는 데에는 충분했다. 소변을 이용한 스마트폰 충전 장치의 시제품은 현재 가격이 수십만원대이다. 연구진은 2년 내 저개발국가에서 쓸 수 있을 정도로 저렴한 장치를 개발할 계획이다. ◇전지용 탄소 촉매도 만들어 유종성 고려대 교수(신소재화학)는 지난 9일 네이처 자매지인 \'사이언티픽 리포트\'지에 소변을 이용한 수소연료전지 촉매를 발표했다. 수소연료전지는 말 그대로 연료로 수소를 쓰는 전지다. 음극에서는 수소가 산화하면서 전자를 내놓고, 양극에서는 산소와 수소, 전자가 만나 물이 된다. 친환경 자동차

In [220]:
df.iloc[1]

id                                                 NIRW1900000024.4104
topic                                                               기술
text                 삼성전자 5대 생활가전, 스마트 가전 시대 주도 삼성전자가 냉장고·에어컨·세탁기·오...
keywords_textrank    ['가전', '제품', '스마트', '전자', '에어컨', '스마트폰', '청소',...
keywords_tfid        ['제품', '가전', '스마트', '전자', '삼성', '스마트폰', '에어컨',...
keywords             {전자, 가전, 기능, 냉장고, 온도, 스마트, 삼성, 생활, 청소, 제품, 로봇,...
match_count                                                          9
match_M_list         {'c12': {'전자'}, 'c26': {'스마트', '전자', '스마트폰'}, ...
Name: 8449, dtype: object

In [221]:
df.iloc[1].text

'삼성전자 5대 생활가전, 스마트 가전 시대 주도 삼성전자가 냉장고·에어컨·세탁기·오븐·청소로봇 등 5대 생활가전 제품에 스마트폰으로 제품을 작동하거나 상태를 확인하는 애플리케이션(앱) 라인업을 모두 갖추고, 스마트 가전 시대를 선도하고 있다. 삼성전자의 5대 가전 제품과 스마트폰은 Wi-Fi를 기반으로 연결되며, 각 제품의 앱은 제품 기능을 작동시키거나 현재 사용하고 있는 제품 상태를 알려 주는 등 제품을 더욱 쉽고 편안하게 쓸 수 있는 기능을 갖추고 있다. 삼성전자 스마트가전 앱은 안드로이드 기반 스마트폰에서 사용 가능하며, 에어컨과 청소로봇 앱은 iOS도 지원한다. 지난달 말 선보인 ‘스마트 냉장고 앱’은 기본적으로 냉장고의 온도를 확인할 수 있고, 냉장실과 냉동실에 저장된 제품의 종류에 따른 유효기간을 알려 주는 푸드 매니저 기능이 있다. 지난해 9월 선보인 ‘세탁기 스마트케어 앱’은 시간표시 부분을 스마트폰에 인식해 빠른 고장진단이 가능하도록 해준다. 지난 1월 선보인 ‘스마트에어컨 Q앱’은 에어컨에 대한 전반적인 상태에 대해 알 수 있을 뿐만 아니라 집 밖에서도 전원과 온도, 운전 모드를 제어할 수 있으며, 날씨 정보를 분석해 냉방·청정·제습 운전을 추천하기도 한다. 지난해 9월 선보인 영상 청소로봇 ‘탱고VIEW 앱’은 원격으로 제품을 조작하고 내장된 카메라로 집안의 내부를 실시간으로 볼 수 있으며, 지난달 선보인 ‘지펠 세라믹 오븐 앱’은 160가지 다양한 레시피 중 골라 오븐에 전송하면 시간과 온도가 자동으로 세팅된다. 삼성전자 생활가전사업부 최구연 상무는 “스마트 가전은 앞으로도 계속 진화할 것”이라며 “향후 삼성 가전제품의 통합 애플리케이션을 준비해 사용자에게 더욱 편리하고 즐거운 삶을 제공할 것”이라고 말했다.'

In [223]:
match_S_list = match_list

In [224]:
df['match_S_list'] = match_S_list

In [225]:
df

,id,topic,text,keywords_textrank,keywords_tfid,keywords,match_count,match_M_list,match_S_list
297158,NWRW1900000006.12652,기술,"스마트폰 충전… 이젠 오줌에 맡겨볼까 소변, 에너지로 부활… 휴대폰 충전·수소車 연...","['소변', '전지', '연료', '탄소', '촉매', '전자', '전기', '구진...","['소변', '연료', '전지', '탄소', '촉매', '전자', '요소', '스마...","{전자, 구진, 연료, 소변, 전기, 요소, 촉매, 탄소, 질소, 전지, 수소, 스...",9,"{'c12': {'전자'}, 'c19': {'연료'}, 'c20': {'수소', '...","{'c120': {'전자'}, 'c192': {'연료'}, 'c201': {'수소'..."
8449,NIRW1900000024.4104,기술,"삼성전자 5대 생활가전, 스마트 가전 시대 주도 삼성전자가 냉장고·에어컨·세탁기·오...","['가전', '제품', '스마트', '전자', '에어컨', '스마트폰', '청소',...","['제품', '가전', '스마트', '전자', '삼성', '스마트폰', '에어컨',...","{전자, 가전, 기능, 냉장고, 온도, 스마트, 삼성, 생활, 청소, 제품, 로봇,...",9,"{'c12': {'전자'}, 'c26': {'스마트', '전자', '스마트폰'}, ...","{'c120': {'전자'}, 'c262': {'스마트', '전자'}, 'c264'..."
297154,NWRW1900000006.12584,기술,스마트폰 충전… 이젠 오줌에 맡겨볼까 머지않아 스마트폰의 배터리가 바닥나면 화장실로...,"['소변', '전지', '연료', '탄소', '촉매', '전자', '전기', '구진...","['소변', '전지', '연료', '촉매', '탄소', '전자', '전기', '요소...","{전자, 구진, 연료, 소변, 전기, 요소, 촉매, 탄소, 질소, 전지, 교수, 스...",8,"{'c12': {'전자'}, 'c19': {'연료'}, 'c20': {'탄소', '...","{'c120': {'전자'}, 'c192': {'연료'}, 'c201': {'탄소'..."
360828,NWRW1900000054.11539,기술,‘아이폰5’ 삼성-애플 대결구도 바꾸나 12일(현지시각) 미국 뉴욕에서 공개될 것으...,"['부품', '전자', '애플', '아이폰', '스마트폰', '삼성', '갤럭시',...","['부품', '전자', '애플', '아이폰', '갤럭시', '삼성', '스마트폰',...","{전자, 경쟁력, 부품, 반도체, 삼성, 메모리, 애플, 갤럭시, 더욱, 스마트폰,...",8,"{'c12': {'전자'}, 'c26': {'전자', '반도체', '애플', '갤럭...","{'c120': {'전자'}, 'c261': {'메모리', '반도체'}, 'c262..."
143759,NPRW1900000058.23991,기술,삼성 스마트폰으로 벤츠 문 여닫아…`IoT 공룡` 출현 ◆ IFA 2016 ◆ 시장...,"['스마트', '냉장고', '센서', '전자', '스마트폰', '허브', '가전',...","['스마트', '냉장고', '전자', '센서', '스마트폰', '서비스', '벤츠'...","{통해, 전자, 가전, 기능, 냉장고, 차량, 벤츠, 스마트, 허브, 서비스, 센서...",8,"{'c12': {'전자'}, 'c26': {'스마트', '전자', '스마트폰', '...","{'c120': {'전자'}, 'c262': {'스마트', '전자', '센서'}, ..."
...,...,...,...,...,...,...,...,...,...
112649,NPRW1900000025.1837,기술,구글이 한글 교육프로그램에 돈 대는 이유는 구글이 외국인을 대상으로 한 온라인 '한...,"['구글', '한글', '문화', '한국', '지원', '박물관', '세계', '콘...","['구글', '한글', '문화', '한국', '지원', '세계', '박물관', '콘...","{장관, 국립, 박물관, 문화, 지원, 세계, 콘텐츠, 프로그램, 한국, 한글, 교...",0,{},{}
112648,NPRW1900000025.1829,기술,"“북한 해커들이 좋아하는 액티브엑스, 감싸면 종북 좌빨” “우리 회사는 관리자의 허...","['액티브엑스', '문제', '공인', '인증서', '결제', '보안', '인터넷'...","['액티브엑스', '문제', '공인', '결제', '보안', '인증서', '인터넷'...","{인증서, 문제, 카드, 액티브엑스, 공인, 마이크로소프트, 보안, 인터넷, 사람,...",0,{},{}
267099,NPRW2200000005.37751,기술,"웅진씽크빅, 학원형클라우드 도전…네이버클라우드와 합작사 설립 네이버클라우드와 웅진씽...","['클라우드', '네이버', '학원', '운영', '제공', '웅진씽크빅', '배컴...","['클라우드', '네이버', '배컴', '학원', '제공', '운영', '사업', ...","{제공, 사교육, 플랫폼, 네이버, 학원, 사업, 웅진씽크빅, 배컴, 운영, 클라우드}",0,{},{}
267101,NPRW2200000005.37753,기술,"일부 알뜰폰 사업자 콜센터 운영 기준 미달…과기정통부, 충원 지시 일부 영세 알뜰폰...","['사업자', '알뜰폰', '이용자', '콜센터', '보호', '점검', '정통부'...","['사업자', '알뜰폰', '이용자', '보호', '콜센터', '정통부', '점검'...","{정통부, 콜센터, 확인, 점검, 알뜰폰, 자체, 사업자, 시행, 가이드라인, 보호...",0,{},{}


In [226]:
max_ksic

[{'c201'},
 {'c285'},
 {'c201', 'c203', 'c262', 'c282'},
 {'c264'},
 {'c262'},
 {'c264'},
 {'c151', 'c264', 'c274', 'c303'},
 {'c262', 'c263', 'c272'},
 {'c264'},
 {'c264'},
 {'c262'},
 {'c264'},
 {'c264'},
 {'c271'},
 {'c151', 'c264', 'c274', 'c333'},
 {'c285'},
 {'c264'},
 {'c285'},
 {'c262', 'c272', 'c273'},
 {'c271'},
 {'c181', 'c264'},
 {'c211', 'c271'},
 {'c172', 'c313'},
 {'c262'},
 {'c201'},
 {'c271'},
 {'c313'},
 {'c264'},
 {'c172', 'c262', 'c264'},
 {'c285'},
 {'c271', 'c334'},
 {'c264'},
 {'c262'},
 {'c313'},
 {'c264'},
 {'c232'},
 {'c285'},
 {'c172', 'c262', 'c264'},
 {'c262', 'c264', 'c273'},
 {'c271', 'c285'},
 {'c264'},
 {'c273'},
 {'c172', 'c222', 'c285'},
 {'c333'},
 {'c264'},
 {'c271'},
 {'c264'},
 {'c262', 'c273'},
 {'c262'},
 {'c264'},
 {'c285'},
 {'c271'},
 {'c262', 'c264'},
 {'c264'},
 {'c264'},
 {'c271'},
 {'c264'},
 {'c262', 'c264'},
 {'c264'},
 {'c264'},
 {'c262', 'c301', 'c303'},
 {'c271'},
 {'c313'},
 {'c285'},
 {'c262', 'c264', 'c273'},
 {'c264'},
 {'c264'},

In [227]:
len(max_ksic)

382513

In [228]:
df['max_S_ksic'] = max_ksic

In [229]:
df[:5]

,id,topic,text,keywords_textrank,keywords_tfid,keywords,match_count,match_M_list,match_S_list,max_S_ksic
297158,NWRW1900000006.12652,기술,"스마트폰 충전… 이젠 오줌에 맡겨볼까 소변, 에너지로 부활… 휴대폰 충전·수소車 연...","['소변', '전지', '연료', '탄소', '촉매', '전자', '전기', '구진...","['소변', '연료', '전지', '탄소', '촉매', '전자', '요소', '스마...","{전자, 구진, 연료, 소변, 전기, 요소, 촉매, 탄소, 질소, 전지, 수소, 스...",9,"{'c12': {'전자'}, 'c19': {'연료'}, 'c20': {'수소', '...","{'c120': {'전자'}, 'c192': {'연료'}, 'c201': {'수소'...",{c201}
8449,NIRW1900000024.4104,기술,"삼성전자 5대 생활가전, 스마트 가전 시대 주도 삼성전자가 냉장고·에어컨·세탁기·오...","['가전', '제품', '스마트', '전자', '에어컨', '스마트폰', '청소',...","['제품', '가전', '스마트', '전자', '삼성', '스마트폰', '에어컨',...","{전자, 가전, 기능, 냉장고, 온도, 스마트, 삼성, 생활, 청소, 제품, 로봇,...",9,"{'c12': {'전자'}, 'c26': {'스마트', '전자', '스마트폰'}, ...","{'c120': {'전자'}, 'c262': {'스마트', '전자'}, 'c264'...",{c285}
297154,NWRW1900000006.12584,기술,스마트폰 충전… 이젠 오줌에 맡겨볼까 머지않아 스마트폰의 배터리가 바닥나면 화장실로...,"['소변', '전지', '연료', '탄소', '촉매', '전자', '전기', '구진...","['소변', '전지', '연료', '촉매', '탄소', '전자', '전기', '요소...","{전자, 구진, 연료, 소변, 전기, 요소, 촉매, 탄소, 질소, 전지, 교수, 스...",8,"{'c12': {'전자'}, 'c19': {'연료'}, 'c20': {'탄소', '...","{'c120': {'전자'}, 'c192': {'연료'}, 'c201': {'탄소'...","{c262, c203, c201, c282}"
360828,NWRW1900000054.11539,기술,‘아이폰5’ 삼성-애플 대결구도 바꾸나 12일(현지시각) 미국 뉴욕에서 공개될 것으...,"['부품', '전자', '애플', '아이폰', '스마트폰', '삼성', '갤럭시',...","['부품', '전자', '애플', '아이폰', '갤럭시', '삼성', '스마트폰',...","{전자, 경쟁력, 부품, 반도체, 삼성, 메모리, 애플, 갤럭시, 더욱, 스마트폰,...",8,"{'c12': {'전자'}, 'c26': {'전자', '반도체', '애플', '갤럭...","{'c120': {'전자'}, 'c261': {'메모리', '반도체'}, 'c262...",{c264}
143759,NPRW1900000058.23991,기술,삼성 스마트폰으로 벤츠 문 여닫아…`IoT 공룡` 출현 ◆ IFA 2016 ◆ 시장...,"['스마트', '냉장고', '센서', '전자', '스마트폰', '허브', '가전',...","['스마트', '냉장고', '전자', '센서', '스마트폰', '서비스', '벤츠'...","{통해, 전자, 가전, 기능, 냉장고, 차량, 벤츠, 스마트, 허브, 서비스, 센서...",8,"{'c12': {'전자'}, 'c26': {'스마트', '전자', '스마트폰', '...","{'c120': {'전자'}, 'c262': {'스마트', '전자', '센서'}, ...",{c262}


In [230]:
max_M_ksic = []

for match in tqdm(match_M_list) :
    max_M_ksic.append(get_max_ksic(match))

100%|████████████████████████████████| 382513/382513 [00:09<00:00, 40313.77it/s]


In [231]:
max_M_ksic

[{'c20', 'c26'},
 {'c26', 'c28'},
 {'c20', 'c26'},
 {'c26'},
 {'c26'},
 {'c26'},
 {'c26'},
 {'c26', 'c27'},
 {'c26'},
 {'c26', 'c28'},
 {'c26'},
 {'c26'},
 {'c26'},
 {'c27'},
 {'c15'},
 {'c26', 'c28'},
 {'c26'},
 {'c28'},
 {'c26', 'c27'},
 {'c27'},
 {'c18', 'c26'},
 {'c27'},
 {'c17', 'c31'},
 {'c26'},
 {'c20'},
 {'c27'},
 {'c31'},
 {'c26'},
 {'c26'},
 {'c28'},
 {'c33'},
 {'c26'},
 {'c26'},
 {'c31'},
 {'c26'},
 {'c23', 'c27'},
 {'c28'},
 {'c26'},
 {'c26'},
 {'c28'},
 {'c26'},
 {'c26', 'c27'},
 {'c17'},
 {'c33'},
 {'c26'},
 {'c27'},
 {'c26'},
 {'c26', 'c27'},
 {'c28'},
 {'c26'},
 {'c28'},
 {'c27'},
 {'c26'},
 {'c26'},
 {'c26'},
 {'c26', 'c27'},
 {'c26'},
 {'c26'},
 {'c26'},
 {'c26'},
 {'c30'},
 {'c27'},
 {'c31'},
 {'c28'},
 {'c26'},
 {'c26'},
 {'c26'},
 {'c26', 'c27'},
 {'c15', 'c26', 'c27'},
 {'c26'},
 {'c26'},
 {'c26'},
 {'c28'},
 {'c26'},
 {'c26'},
 {'c18', 'c28'},
 {'c28'},
 {'c26', 'c30'},
 {'c26', 'c27'},
 {'c26'},
 {'c28'},
 {'c26', 'c27'},
 {'c26'},
 {'c26'},
 {'c17'},
 {'c26'},


In [234]:
max_count = 0
max_idx = 0

for index, ksic in enumerate(max_M_ksic[:1000]) :
    if max_count < len(ksic):
        max_count = len(ksic)
        max_idx = index

In [242]:
df['max_M_ksic'] = max_M_ksic

In [235]:
max_count

9

In [236]:
max_idx

275

In [237]:
max_M_ksic[max_idx]

{'c12', 'c13', 'c17', 'c26', 'c27', 'c28', 'c30', 'c31', 'c33'}

In [244]:
row = df.iloc[275]

In [245]:
row

id                                                  NWRW1900000039.131
topic                                                               기술
text                 미세먼지 잡는다는 차량 에어컨 필터, 알고보니 엉터리 공정거래위원회는 차량 에어컨필...
keywords_textrank    ['필터', '에어컨', '광고', '한국', '쓰리엠', '차량', '표시', '...
keywords_tfid        ['필터', '에어컨', '광고', '차량', '한국', '항균', '표시', '제...
keywords             {전자, 과징금, 차량, 필터, 표시, 쓰리엠, 한국, 광고, 항균, 제품, 에어컨...
match_count                                                          6
match_M_list         {'c12': {'전자'}, 'c13': {'필터'}, 'c17': {'필터'}, ...
match_S_list         {'c120': {'전자'}, 'c139': {'필터'}, 'c179': {'필터'...
max_S_ksic           {c262, c285, c312, c179, c271, c339, c303, c13...
max_M_ksic               {c30, c12, c33, c27, c17, c26, c31, c28, c13}
Name: 341898, dtype: object

In [246]:
row.max_S_ksic

{'c120',
 'c139',
 'c179',
 'c262',
 'c271',
 'c285',
 'c303',
 'c312',
 'c319',
 'c339'}

In [247]:
row.max_M_ksic

{'c12', 'c13', 'c17', 'c26', 'c27', 'c28', 'c30', 'c31', 'c33'}

In [248]:
row.text

'미세먼지 잡는다는 차량 에어컨 필터, 알고보니 엉터리 공정거래위원회는 차량 에어컨필터 성능을 과장 표시·광고한 한국쓰리엠 등 4개 차량 에어컨필터 제조사에 과징금 2100만원을 부과하기로 결정했다고 2일 밝혔다. 한국쓰리엠이 1700만원, 두원전자가 400만원의 과징금을, 에이펙코리아·엠투는 각각 시정명령을 받았다. 차량 에어컨필터는 차량 외부에서 실내로 들어오는 공기를 정화하는 실내공기정화필터로, 자동차 에어컨·히터 장치에 장착된다. 공정위 조사결과 이들은 객관적·과학적으로 입증되지 않은 미세먼저 제거효능을 과장해 광고한 것으로 나타났다. 두원전자는 2012년부터 지난해까지 115종 차량 에어컨필터를 판매하면서 ‘청정효율 : 2～5㎛ 70%이상’이라고 표시했다. 한국쓰리엠과 에이펙코리아도 ‘5미크론 이상 입자 제거효율 99%’, ‘미세먼지를 완벽하게 걸러줍니다’ 등의 표현을 사용했다. 이들은 사업자가 광고 내용을 합리적 근거 등으로 증명할 의무를 명시한 표시광고법을 지키지 못한 것으로 조사됐다.항균효과 광고도 과장된 것으로 밝혀졌다. 한국쓰리엠과 엠투는 일부 에어컨필터 제품을 팔면서 ‘항균정전필터’, ‘에어컨 내부 세균번식을 억제’, ‘뛰어난 항균력’ 등을 광고했으나 이를 입증하는 데는 실패했다. 두원전자는 FITI시험연구원에서 SF인증 얻지 못했지만 인터넷 쇼핑몰 등에서 SF마크를 허위로 표시했다가 적발됐다. SF마크는 FITI시험연구원이 항균 등 위생가공처리한 제품을 제조·판매하는 업체를 대상으로 공장심사 및 제품시험을 실시하여 합격한 제품에 부여하는 인증마크다.공정위는 이들 업체의 관련 매출액에 1.4% 과징금 부과율을 적용했다. 에이펙코리아와 엠투는 중소업체로 위반정도가 경미하다는 이유로 과징금을 부과하지 않았다.'

In [249]:
row.match_S_list

{'c120': {'전자'},
 'c139': {'필터'},
 'c179': {'필터'},
 'c262': {'전자'},
 'c271': {'항균'},
 'c285': {'에어컨'},
 'c303': {'차량'},
 'c312': {'차량'},
 'c319': {'차량'},
 'c339': {'광고'}}

In [252]:
row.keywords

{'과징금', '광고', '쓰리엠', '에어컨', '에이펙', '전자', '제품', '차량', '표시', '필터', '한국', '항균'}

In [253]:
row.keywords_tfid

"['필터', '에어컨', '광고', '차량', '한국', '항균', '표시', '제품', '쓰리엠', '과징금']"

In [254]:
row.keywords_textrank

"['필터', '에어컨', '광고', '한국', '쓰리엠', '차량', '표시', '전자', '항균', '에이펙']"